In [1]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import numpy as np
import random
import pickle
from collections import Counter

In [2]:
lemmatizer=WordNetLemmatizer()
hm_lines=100000000

In [3]:
def create_lexicon(pos,neg):
    lexicon=[]
    for fi in [pos,neg]:
        with open(fi,'r') as f:
            contents=f.readlines()
            for l in contents[:hm_lines]:
                all_words=word_tokenize(l.lower())
                #print(all_words)
                lexicon+=list(all_words)
    lexicon=[lemmatizer.lemmatize(i) for i in lexicon]
    w_counts=Counter(lexicon)
    #print(w_counts)
    #w_counts={'the':4657,'a':3737}....etc
    l2=[]
    for w in w_counts:
        if 1000> w_counts[w] >50:
            l2.append(w)
    #print(len(l2))
    #print(l2)
    return l2

In [4]:
def sample_handling(sample, lexicon, classification):
    featureset=[]
    with open(sample,'r') as f:
        contents= f.readlines()
        for l in contents[:hm_lines]:
            current_words=word_tokenize(l.lower())
            #print(current_words)
            current_words=[lemmatizer.lemmatize(i) for i in current_words]
            #print(current_words)
            features=np.zeros(len(lexicon))
            for word in current_words:
                if word.lower() in lexicon:
                    index_value=lexicon.index(word.lower())
                    features[index_value]+=1
            features=list(features)
            featureset.append([features, classification])
    #print(featureset[:10])
    return featureset

In [5]:
def create_feature_sets_and_labels(pos,neg,test_size=0.1):
    lexicon=create_lexicon(pos,neg)
    features=[]
    features+=sample_handling(pos,lexicon,[1,0])
    features+=sample_handling(neg,lexicon,[0,1])
    random.shuffle(features)
    
    features=np.array(features)
    testing_size=int(test_size*len(features))
    
    train_x=list(features[:,0][:-testing_size])
    train_y=list(features[:,1][:-testing_size])
    test_x=list(features[:,0][-testing_size:])
    test_y=list(features[:,1][-testing_size:])
    
    return train_x,train_y,test_x,test_y

In [6]:
if __name__ == '__main__':
    train_x,train_y,test_x,test_y=create_feature_sets_and_labels('pos.txt','neg.txt')
    with open("sentiments set.pickle","wb") as f:
        pickle.dump([train_x,train_y,test_x,test_y],f)
        